# Scrapping Básico - Descargando páginas web

El primer paso antes de hacer cualquier tipo de scrapping es decidir que página queremos descargar y analizar. En nuestro caso vamos a obtener datos de la página del Instituto Carlos III sobre la evolución del COVID-19:

https://covid19.isciii.es/

Lo primero que deberíamos hacer sería visualizar la página en nuestro navegador favorito (¿Firefox?) para comprobar el aspecto que tiene y la información que podemos descargar.

A continuación tendremos que emular el comportamiento del navegador para obtener la página: tendremos que hacer una petición HTTP para recuperar la página web. Lo haremos usando la biblioteca "requests".

Información y ejemplos sobre dicho módulo: https://www.w3schools.com/python/module_requests.asp

In [ ]:
import requests
from pprint import pprint

respuesta = requests.get("https://covid19.isciii.es/")  # Utilizamo el metodo GET del protocolor HTTP
                                                        

In [ ]:
# Hemos obtenido un objeto de tipo "Response", donde estará toda la información que hemos obtenido de la página
# https://www.w3schools.com/python/ref_requests_response.asp

pprint(vars(respuesta))  # Podemos inspeccionar el objeto y ver que campos nos interesan. Particularmente:


In [ ]:
print(respuesta.status_code)   # El código de estado: 200 Es el código de que todo ha ido bien. 
                               # Lista de códigos de estados posibles: https://www.restapitutorial.com/httpstatuscodes.html
    
print(respuesta.reason)        # Una descripción textual del código de respuesta  


print(respuesta.elapsed)       # Tiempo que se ha tardado desde que se hizo la petición hasta que se obtuvo la respuesta


print(respuesta.encoding)      # La codificación del documento (juego de caracteres usado)


In [ ]:
print(respuesta.content)     # Y lo que probablemente nos interesa más: el CONTENIDO de la página.
                             # El "problema" es que es una cadena "binaria".

In [ ]:
html = respuesta.content.decode("UTF-8")   # Lo podemos codificar a una cadena "normal"

print(html)


In [ ]:
# Podemos guardar el código HTML a un fichero local para poder inspeccionarlo con más tranquilidad

text_file = open("covid-19.html", "w")
text_file.write(html)
text_file.close()

In [ ]:
# También puede ser interesante usar el iterador que te permite recorrer el contenido obtenido línea a línea
for num, linea in enumerate(respuesta.iter_lines(), start=1):
  print("Línea %d: %s" % (num, linea.decode("UTF-8")) )

## Bajando ficheros binarios

Normalmente las páginas web están escritas en ```HTML``` que es tipo de fichero de texto (string) fácil de manejar. Sin embargo en ocasiones querremos bajar ficheros que estén en otros formatos que no sean texto (binarios), como son los formatos ```PDF```, ```EXE```, ```ZIP```, ```JPG```.

En una sección anterior del curso se planteó un script para bajar el BOP de Granada y luego sacar información del ```PDF```. A continuación vamos a "refinar" el script que se creó simplificando y aprovechando alguna información que nos ofrece el protocolo ```HTTP```.


In [ ]:
import datetime
import requests

baseURL="http://bop2.dipgra.es:8880/opencms/opencms/portal/DescargaPDFBoletin?fecha="

fecha = datetime.datetime.now()
#fecha = datetime.datetime(2020, 5, 6)  # Año, mes, día, para una fecha que no sea hoy
print(fecha)

fechaFormateada = fecha.strftime("%d/%m/%Y")  # Formateamos la fecha, https://www.programiz.com/python-programming/datetime/strftime
print(fechaFormateada)

url = baseURL + fechaFormateada
print(url)

In [ ]:
respuesta = requests.get(url)  # hacemos la petición y se descargan los contenidos


In [ ]:
print("Cabeceras recibidas: " + str(respuesta.headers) + "\n") # Inspeccionamos las cabeceras que hemos recibido

# Nos interesa ver que "Content-Type" ya nos indica que es un fichero que puede abrirse con una aplicación 
# que maneje PDFs
# Además en alguna ocasión podemos recibir que el tamaño del contenido descargado: "Content-Length"

print("Tipo de información descargada: " + respuesta.headers['Content-Type'] + "\n")

tamanioDescargado = 0
if "Content-Length" in respuesta.headers:
    tamanioDescargado = int(respuesta.headers['Content-Length'])
else:
    tamanioDescargado = len(respuesta.content)  # Si no nos lo dice la cabecera podemos mirar el tamaño del
                                                # contenido descargado
        
    
print("Tamaño descargado: " + str(tamanioDescargado) + " bytes\n")

# Además nos damos cuenta que la aplicación web nos sugiere un nombre para el fichero que vamos a descargar:

print("Campo que nos sugiere el nombre del fichero descargado: " + respuesta.headers['Content-disposition'] + "\n")

# En la aplicación web que sirve los BOP de Granada siempre devuelve una respuesta válida (código 200)
# aunque no exista el PDF de la fecha pedida. En una aplicación bien hecha devolvería un codigo de error distinto
# (NOT FOUND o similar). En cualquier caso nosotros nos hemos dado cuenta que si el contenido devuelto es 0 bytes
# es que ese día no hay BOP.

if tamanioDescargado == 0:
    print("En la fecha indicada no hay BOP (o es una fecha incorrecta).\n")
else:
    
    filename = respuesta.headers['Content-disposition'][20:] # Saco el nombre del fichero truncando la cadena
    
    myfile = open(filename,'wb')
    myfile.write(respuesta.content)
    myfile.close()
    
    print("Fichero " + filename + " grabado en disco.\n")